In [1]:
class Quantity: #1
    def __init__(self, storage_name):
        self.storage_name = storage_name #2
    
    def __set__(self, instance, value): #3
        if value > 0:
            instance.__dict__[self.storage_name] = value #4
        else:
            raise ValueError('value must be > 0')
            

class LineItem:
    weight = Quantity('weight') #5
    price = Quantity('price') #6
    
    def __init__(self, description, weight, price): #7
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [2]:
truffle = LineItem('White truffle', 100, 0)

ValueError: value must be > 0

In [18]:
class Quantity:
    __counter = 0 #1
    
    def __init__(self):
        cls = self.__class__ #2
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index) #3
        cls.__counter += 1 #4
        
    def __get__(self, instance, owner): #5
        return getattr(instance, self.storage_name) #6
    
    def __set__(self, instance, value): #6
        if value > 0:
            setattr(instance, self.storage_name, value) #7
        else:
            raise ValueError('value must be > 0')
            
class LineItem:
    weight = Quantity() #9
    price = Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [19]:
coconuts = LineItem('Brazilian coconut', 20, 17.95)

In [20]:
coconuts.weight, coconuts.price

(20, 17.95)

In [21]:
getattr(coconuts, '_Quantity#0')

20

In [24]:
LineItem.weight

In [23]:
class Quantity:
    __counter = 0 #1
    
    def __init__(self):
        cls = self.__class__ #2
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index) #3
        cls.__counter += 1 #4
        
    def __get__(self, instance, owner): #5
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name) #6
    
    def __set__(self, instance, value): #6
        if value > 0:
            setattr(instance, self.storage_name, value) #7
        else:
            raise ValueError('value must be > 0')
            
class LineItem:
    weight = Quantity() #9
    price = Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [ ]:
import model_v4c as model


class LineItem:
    weight = model.Quantity()
    price = model.Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [25]:
def quantity(): #1
    try:
        quantity.counter += 1 #2
    except AttributeError:
        quantity.counter = 0 #3
    storage_name = '_{}:{}'.format('quntity', quantity.counter) #4
    
    def qty_getter(instance): #5
        return getattr(instance, storage_name)
    
    def qty_setter(instance, value):
        if value > 0:
            setattr(instance, storage_name, value)
        else:
            raise ValueError('value must be > 0')
    
    return property(qty_getter, qty_setter)